# Imports and Initial Formatting

In [1]:
import pandas as pd
data = pd.read_csv('../resources/betting_odds.csv')
data = data[['Date','VH','Team','Final','Close','ML']]
data = data.replace('pk', '0')

# Pulling Data for DataFrame

In [2]:
date = []
home = []
score_h = []
score_a = []
away = []
ml_h = []
ml_a = []
either = []
for column, row in data.iterrows():
    if column % 2 == 0:
        date.append(row['Date'])
        away.append(row['Team'])
        score_a.append(row['Final'])
        ml_a.append(row['ML'])
    else:
        home.append(row['Team'])
        score_h.append(row['Final'])
        ml_h.append(row['ML'])
    either.append(row['Close'])

# Parses Over/Under and Spread

In [3]:
spread = []
ou = []
i = 0
for x in range(0,int((len(either))/2),1):
    first = float(either[i])
    i += 1
    second = float(either[i])
    i += 1
    if first > second:
        ou.append(first)
        spread.append(second)
    else: 
        ou.append(second)
        spread.append(first)

# Creates Formatted DataFrame for Analysis

In [4]:
data = pd.DataFrame({'date':date,'ml_a':ml_a,'away':away,'score_a':score_a,'score_h':score_h,
                      'home':home,'ml_h':ml_h,'o/u':ou,'spread':spread})
data = data.dropna(how='any')

# Splitting Data and Adding ATS Columns

In [5]:
cover = []
result = []
for index, game in data.iterrows():
    # Home Favorite
    if game['ml_h'] < game['ml_a']:
        if (game['score_h'] - game['spread']) > game['score_a']:
            cover.append('Home Favorite Covers')
        elif (game['score_h'] - game['spread']) < game['score_a']:
            cover.append('Away Underdog Covers')
        else:
            cover.append('HF / AU Equals')
            
        if game['score_h'] > game['score_a']:
            result.append('Home Favorite Win')
        elif game['score_h'] < game['score_a']:
            result.append('Away Underdog Win')
        else:
            result.append('HF / AU Tie')
    # Away Favorite
    elif game['ml_h'] > game['ml_a']:
        if (game['score_a'] - game['spread']) > game['score_h']:
            cover.append('Away Favorite Covers')
        elif (game['score_a'] - game['spread']) < game['score_h']:
            cover.append('Home Underdog Covers')
        else:
            cover.append('HU / AF Equals')
            
        if game['score_h'] > game['score_a']:
            result.append('Home Underdog Win')
        elif game['score_h'] < game['score_a']:
            result.append('Away Favorite Win')
        else:
            result.append('HU / AF Tie')
    # No Favorite
    else: 
        if game['score_h'] > game['score_a']:
            cover.append('Home Even Win')
        elif game['score_h'] < game['score_a']:
            cover.append('Away Even Win')
        else:
            cover.append('Tie Equals')
            
        if game['score_h'] > game['score_a']:
            result.append('Home Win')
        elif game['score_h'] < game['score_a']:
            result.append('Away Win')
        else:
            result.append('Tie')
data['cover'] = cover
data['result'] = result

# Exporting Formatted File

In [6]:
data.to_csv('../resources/reformatted.csv')